In [ ]:
def get_filepath_target(data, Date, image_path_location):

  All_filepath=[]
  All_target=[]
  All_location=[]
  All_Ninfo=[]
  for Rownumber in range(data.shape[0]):
    location=data.loc[Rownumber]['location']
    row=data.loc[Rownumber]['row']
    rangeno=data.loc[Rownumber]['range']
    experiment=data.loc[Rownumber]['experiment']
    genotype=data.loc[Rownumber]['genotype']
    groundtruth=data.loc[Rownumber][13:].to_string(index=True)

    # print('\n')
    #print('if NaN, it means we have no data.')
    #print('\n')
    #print(f'The experimental plot with genotype name {genotype} was at: {location} grown with experimental code in column "experiment" in dataframe: {experiment} in range {rangeno} and row {row}. The ground truth data available are:')

    #print('\n')
    #print(groundtruth)
    #print('\n')
    #print('Image file names belonging to the plot and their relative paths are:')


    Final_path=[]

    for files in image_path_location[0]:
      # print(files)
      if not files==location:
          continue
      finallocation=location
      finalimagefolder=[os.path.join(x,finallocation) for x in paths_to_image_files]


    for locationfolder in finalimagefolder:

      timepointfolder=os.listdir(locationfolder)
      timepointfolder=sorted(timepointfolder)
      imagetype=locationfolder.split('/')[1]
      # print(imagetype)

      timepointpath=[os.path.join(locationfolder,x) for x in timepointfolder]

      for timepointpath_ in timepointpath:

          imagefiles=os.listdir(timepointpath_)

          for images in imagefiles:
              range_=images.split('_')[1]
              row_=images.split('_')[2].split('.')[0]
              experiment_=images.split('_')[0].split('-')[2]

              # print(range_, row_,experiment_)
              # print(rangeno, row, experiment)
              if str(range_)==str(rangeno) and str(row_)==str(row) and str(experiment)==str(experiment_):

                  timepoint=images.split('_')[0].split('-')[1]

                  date_=Date.loc[(Date['Location']==location) & (Date['Image']==imagetype) & (Date['time']==timepoint)]['Date'].to_string(index=False)
                  finalpath=os.path.join(timepointpath_,images)
                  print(f'{finalpath}, Timepoint of image {timepoint} = {date_}')

                  Final_path.append(finalpath)

    All_filepath.append(Final_path)
    All_target.append(data.loc[Rownumber]['yieldPerAcre'])
    All_location.append(data.loc[Rownumber]['location'])
    All_Ninfo.append(data.loc[Rownumber]['poundsOfNitrogenPerAcre'])
  return All_filepath, All_target, All_location, All_Ninfo

In [ ]:
img_height=11
img_width=21
num_channels=6
batch_size=32

def load_image(file_path):
    """Load a raster image into a numpy array and resize it."""
    with rasterio.open(file_path) as src:
        # Read all bands (channels) into a 3D numpy array
        image = np.stack([src.read(i + 1) for i in range(src.count)], axis=-1)

    # Resize image if necessary
    if image.shape[:2] != (img_height, img_width):
        image = tf.image.resize(image, [img_height, img_width])
        image = image.numpy()  # Convert Tensor back to numpy array

    return image

def average_images(file_paths):
    """Compute the average of multiple images."""
    # Load the first image to get its shape
    first_image = load_image(file_paths[0])

    # Initialize an array to hold the sum of images
    sum_image = np.zeros_like(first_image, dtype=np.float64)

    # Load and accumulate the images
    for file_path in file_paths:
        image = load_image(file_path)
        sum_image += image

    # Compute the average
    avg_image = sum_image / len(file_paths)
    # Resize the image if necessary
    avg_image = tf.image.resize(avg_image, [img_height, img_width])
     # Normalize pixel values (Adjust based on your data)
    avg_image = avg_image / 4096.0
    return avg_image

def image_and_target_generator(file_paths, locations, nitrogen, targets):
    """Generate images and targets from file paths and target values."""
    for file_path, loc, nitrogen, target in zip(file_paths, locations, nitrogen, targets):
        if not file_path:
            # Skip empty file paths
            continue
        # print(file_path, target)
        image = average_images(file_path)
        yield image, loc, nitrogen, target

In [ ]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/Competition/Data Unzipped/Dataset_withmoreinfo'
dataset = tf.data.experimental.load(save_directory)

In [ ]:
# Inspect the dataset
for example in dataset.take(1):
    image, location, nitrogen, labels = example
    print(f"Image shape: {image.shape}")
    print(f"Location shape: {location.shape}")
    print(f"Nitrogen shape: {nitrogen.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Image sample value: {image[0, ..., 0].numpy()}")

Image shape: (32, 11, 21, 6)
Location shape: (32, 4)
Nitrogen shape: (32,)
Labels shape: (32,)
Indices sample value: [[0.11294463 0.13156697 0.16570789 0.18151979 0.18685794 0.2001904
  0.21652068 0.21743207 0.21491702 0.21561669 0.2166378  0.19652332
  0.17491652 0.1383865  0.1112022  0.12607346 0.1399058  0.1509677
  0.15507245 0.14700225 0.14260031]
 [0.14357688 0.15812597 0.1847993  0.18607092 0.17282726 0.1752072
  0.18344596 0.17826334 0.1700982  0.16940348 0.16945578 0.16957785
  0.16746406 0.14299189 0.12467044 0.13402723 0.143579   0.15365078
  0.15582196 0.14416716 0.13780999]
 [0.15164831 0.16037291 0.176368   0.17784657 0.17103001 0.18379122
  0.20389406 0.20805009 0.20866238 0.2070781  0.20527416 0.1868183
  0.16701207 0.13370238 0.11016504 0.1306031  0.14727177 0.15388875
  0.1543011  0.14385538 0.1381577 ]
 [0.15247323 0.15920453 0.17154525 0.17532997 0.17422552 0.18865448
  0.20890847 0.20460093 0.19483532 0.18751815 0.18044582 0.18387347
  0.18403938 0.15158807 0.12622

In [ ]:
def compute_vegetation_indices(image):
    R = image[..., 0]
    G = image[..., 1]
    B = image[..., 2]
    NIR = image[..., 3]
    RE = image[..., 4]

    NDVI = (NIR - R) / (NIR + R)
    EVI = 2.5 * ((NIR - R) / (NIR + 6 * R - 7.5 * B + 1))
    SAVI = 1.5 * (NIR - R) / (NIR + R + 0.5)
    GLI = (2 * G - R - B) / (2 * G + R + B)
    NDRE = (NIR - RE) / (NIR + RE)
    GNDVI = (NIR - G) / (NIR + G)
    NGRDI = (R - G) / (R + G)

    # Handle NaNs
    NDVI = tf.where(tf.math.is_nan(NDVI), tf.zeros_like(NDVI), NDVI)
    EVI = tf.where(tf.math.is_nan(EVI), tf.zeros_like(EVI), EVI)
    SAVI = tf.where(tf.math.is_nan(SAVI), tf.zeros_like(SAVI), SAVI)
    GLI = tf.where(tf.math.is_nan(GLI), tf.zeros_like(GLI), GLI)
    NDRE = tf.where(tf.math.is_nan(NDRE), tf.zeros_like(NDRE), NDRE)
    GNDVI = tf.where(tf.math.is_nan(GNDVI), tf.zeros_like(GNDVI), GNDVI)
    NGRDI = tf.where(tf.math.is_nan(NGRDI), tf.zeros_like(NGRDI), NGRDI)


    # Calculate mean for each vegetation index across the spatial dimensions
    mean_NDVI = tf.reduce_mean(NDVI, axis=[1, 2])
    mean_EVI = tf.reduce_mean(EVI, axis=[1, 2])
    mean_SAVI = tf.reduce_mean(SAVI, axis=[1, 2])
    mean_GLI = tf.reduce_mean(GLI, axis=[1, 2])
    mean_NDRE = tf.reduce_mean(NDRE, axis=[1, 2])
    mean_GNDVI = tf.reduce_mean(GNDVI, axis=[1, 2])
    mean_NGRDI = tf.reduce_mean(NGRDI, axis=[1, 2])

    return mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI

def replace_images_with_indices(image, location, nitrogen, labels):
    mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI = compute_vegetation_indices(image)
    return image, location, nitrogen, mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI, labels

def prepare_dataset_for_training(dataset):
    dataset = dataset.map(replace_images_with_indices)
    return dataset

# Apply to your dataset
dataset_with_indices = prepare_dataset_for_training(dataset) # Example to verify shapes
for example in dataset_with_indices.take(1):
    image, location, nitrogen, mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI, labels = example
    print(f"Image shape: {image.shape}")
    print(f"Location shape: {location.shape}")
    print(f"Nitrogen shape: {nitrogen.shape}")
    print(f"Mean NDVI shape: {mean_NDVI.shape}")
    print(f"Mean EVI shape: {mean_EVI.shape}")
    print(f"Mean SAVI shape: {mean_SAVI.shape}")
    print(f"Mean GLI shape: {mean_GLI.shape}")
    print(f"Mean NDRE shape: {mean_NDRE.shape}")
    print(f"Mean GNDVI shape: {mean_GNDVI.shape}")
    print(f"Mean NGRDI shape: {mean_NGRDI.shape}")
    print(f"Labels shape: {labels.shape}")


Image shape: (32, 11, 21, 6)
Location shape: (32, 4)
Nitrogen shape: (32,)
Mean NDVI shape: (32,)
Mean EVI shape: (32,)
Mean SAVI shape: (32,)
Mean GLI shape: (32,)
Mean NDRE shape: (32,)
Mean GNDVI shape: (32,)
Mean NGRDI shape: (32,)
Labels shape: (32,)


In [ ]:
# Map the dataset to combine inputs
def map_dataset(image, location, nitrogen, mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI, target):
    return (image, location, nitrogen, mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI), target

mapped_dataset = dataset_with_indices.map(map_dataset)

# Count the total number of batches in the dataset
total_batches = sum(1 for _ in mapped_dataset)
train_batches = int(0.6 * total_batches)
val_batches = int(0.2 * total_batches)
test_batches = total_batches - train_batches - val_batches

# Split the dataset based on batches
train_dataset = mapped_dataset.take(train_batches)
val_dataset = mapped_dataset.skip(train_batches).take(val_batches)
test_dataset = mapped_dataset.skip(train_batches + val_batches)

#  Check the number of batches in each split
print(f"Training dataset batches: {sum(1 for _ in train_dataset)}")
print(f"Validation dataset batches: {sum(1 for _ in val_dataset)}")
print(f"Test dataset batches: {sum(1 for _ in test_dataset)}")

Training dataset batches: 49
Validation dataset batches: 16
Test dataset batches: 17


In [ ]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define RMSE as a custom metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Define R² as a custom metric
def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

# Define the image input branch
image_input = layers.Input(shape=(11, 21, 6), name='image_input')
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)

# Define the location input branch
location_input = layers.Input(shape=(4,), name='location_input')
y = layers.Dense(16, activation='relu')(location_input)
y = layers.Dense(8, activation='relu')(y)  # New layer after the existing one
y = layers.Dense(4, activation='relu')(y)  # New layer after the existing one

# Define the nitrogen input branch
nitrogen_input = layers.Input(shape=(1,), name='nitrogen_input')
z = layers.Dense(16, activation='relu')(nitrogen_input)
z = layers.Dense(8, activation='relu')(z)  # New layer after the existing one
z = layers.Dense(4, activation='relu')(z)  # New layer after the existing one


# Define separate inputs for vegetation indices
ndvi_input = layers.Input(shape=(1,), name='ndvi_input')
evi_input = layers.Input(shape=(1,), name='evi_input')
savi_input = layers.Input(shape=(1,), name='savi_input')
gli_input = layers.Input(shape=(1,), name='gli_input')
ndre_input = layers.Input(shape=(1,), name='ndre_input')
gndvi_input = layers.Input(shape=(1,), name='gndvi_input')
ngrdi_input = layers.Input(shape=(1,), name='ngrdi_input')

# Process each vegetation index input
ndvi_processed = layers.Dense(8, activation='relu')(ndvi_input)
evi_processed = layers.Dense(8, activation='relu')(evi_input)
savi_processed = layers.Dense(8, activation='relu')(savi_input)
gli_processed = layers.Dense(8, activation='relu')(gli_input)
ndre_processed = layers.Dense(8, activation='relu')(ndre_input)
gndvi_processed = layers.Dense(8, activation='relu')(gndvi_input)
ngrdi_processed = layers.Dense(8, activation='relu')(ngrdi_input)

# Concatenate all branches
concatenated = layers.concatenate([x, y, z, ndvi_processed, evi_processed, savi_processed, gli_processed, ndre_processed, gndvi_processed, ngrdi_processed])
output = layers.Dense(1)(concatenated)  # Output layer for regression

optimizer = Adam(learning_rate=0.001)
# Create the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=1e-6)

# Create and compile the model
model = models.Model(inputs=[image_input, location_input, nitrogen_input, ndvi_input, evi_input, savi_input, gli_input, ndre_input, gndvi_input, ngrdi_input], outputs=output)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[rmse, r_squared])

# Print the model summary
model.summary()

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_rmse',  # Monitor the validation RMSE
    patience=50,         # Stop after 50 epochs with no improvement
    restore_best_weights=True,  # Restore the weights of the best epoch
    mode='min'           # RMSE should be minimized
)

# Train the model with the EarlyStopping callback
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=400
)

# Evaluate the model
test_loss, test_rmse, test_r_squared = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test RMSE: {test_rmse}")
print(f"Test R-squared: {test_r_squared}")


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 11, 21, 6)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_38 (Conv2D)        │ (None, 9, 19, 32)      │          1,760 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_38          │ (None, 4, 9, 32)       │              0 │ conv2d_38[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 2, 7, 64)       │         18,496 │ max_pooling2d_38[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ location_input            │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ nitrogen_input            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_39          │ (None, 1, 3, 64)       │              0 │ conv2d_39[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_258 (Dense)         │ (None, 16)             │             80 │ location_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_261 (Dense)         │ (None, 16)             │             32 │ nitrogen_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_19 (Flatten)      │ (None, 192)            │              0 │ max_pooling2d_39[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_259 (Dense)         │ (None, 8)              │            136 │ dense_258[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_262 (Dense)         │ (None, 8)              │            136 │ dense_261[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ndvi_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ evi_input (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ savi_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gli_input (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ndre_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gndvi_input (InputLay

 Total params: 70,553 (275.60 KB)

 Trainable params: 70,553 (275.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 13818.5938 - r_squared: -655.4695 - rmse: 107.7942 - val_loss: 2908.4873 - val_r_squared: -118.4364 - val_rmse: 51.8783
Epoch 2/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2880.7324 - r_squared: -157.1790 - rmse: 50.5814 - val_loss: 2872.5383 - val_r_squared: -116.5564 - val_rmse: 51.5246
Epoch 3/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2897.0364 - r_squared: -161.8098 - rmse: 50.2688 - val_loss: 2800.5667 - val_r_squared: -113.6248 - val_rmse: 50.8288
Epoch 4/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2852.7673 - r_squared: -159.4442 - rmse: 49.9206 - val_loss: 2740.7148 - val_r_squared: -111.0422 - val_rmse: 50.2340
Epoch 5/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2824.4844 - r_squared: -158.3904 - rmse: 49.6313 - val_loss: 2729.9138 - val_r_squared: -109.9412 - val_rmse: 50.1374
Epoch 6/400
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2791.6238 - r_squared: -156.5247 - rmse: 49.4064 - v

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Competition/Submitted_models/'+'cnn_regressionwithmoreinfo_modelNDVIfea_v2_S22.h5')

In [ ]:
test_path='/content/drive/MyDrive/Competition/Data Unzipped/test/Test'

path_to_files=os.listdir(test_path)
paths_to_image_files=[os.path.join(test_path,x) for x in path_to_files if x=='Satellite']
image_path_location=[os.listdir(x) for x in paths_to_image_files]
print(image_path_location[0])

print(image_path_location,paths_to_image_files)
groundtruth_path1= os.path.join(test_path,'GroundTruth/test_HIPS_HYBRIDS_2023_V2.3.csv')
groundtruth_path2= os.path.join(test_path,'GroundTruth/DateofCollection.xlsx')
data=pd.read_csv(groundtruth_path1)
Date=pd.read_excel(groundtruth_path2)

Test_All_Path, Test_All_target, Test_All_loc, Test_All_Ninfo = get_filepath_target(data, Date, image_path_location)

['.DS_Store', 'Ames']
[['.DS_Store', 'Ames']] ['/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite']
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP1/Ames-TP1-2315_9_15.TIF, Timepoint of image TP1 = Series([], )
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP2/Ames-TP2-2315_9_15.TIF, Timepoint of image TP2 = Series([], )
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP3/Ames-TP3-2315_9_15.TIF, Timepoint of image TP3 = Series([], )
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP1/Ames-TP1-2315_7_11.TIF, Timepoint of image TP1 = Series([], )
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP2/Ames-TP2-2315_7_11.TIF, Timepoint of image TP2 = Series([], )
/content/drive/MyDrive/Competition/Data Unzipped/test/Test/Satellite/Ames/TP3/Ames-TP3-2315_7_11.TIF, Timepoint of image TP3 = Series([], )
/content/drive/MyDrive/Competition/Data U

In [ ]:
def compute_vegetation_indices(image):
    R = image[..., 0]
    G = image[..., 1]
    B = image[..., 2]
    NIR = image[..., 3]
    RE = image[..., 4]

    NDVI = (NIR - R) / (NIR + R)
    EVI = 2.5 * ((NIR - R) / (NIR + 6 * R - 7.5 * B + 1))
    SAVI = 1.5 * (NIR - R) / (NIR + R + 0.5)
    GLI = (2 * G - R - B) / (2 * G + R + B)
    NDRE = (NIR - RE) / (NIR + RE)
    GNDVI = (NIR - G) / (NIR + G)
    NGRDI = (R - G) / (R + G)


    # Handle NaNs
    NDVI = tf.where(tf.math.is_nan(NDVI), tf.zeros_like(NDVI), NDVI)
    EVI = tf.where(tf.math.is_nan(EVI), tf.zeros_like(EVI), EVI)
    SAVI = tf.where(tf.math.is_nan(SAVI), tf.zeros_like(SAVI), SAVI)
    GLI = tf.where(tf.math.is_nan(GLI), tf.zeros_like(GLI), GLI)
    NDRE = tf.where(tf.math.is_nan(NDRE), tf.zeros_like(NDRE), NDRE)
    GNDVI = tf.where(tf.math.is_nan(GNDVI), tf.zeros_like(GNDVI), GNDVI)
    NGRDI = tf.where(tf.math.is_nan(NGRDI), tf.zeros_like(NGRDI), NGRDI)

    # Calculate mean for each vegetation index across the spatial dimensions
    mean_NDVI = np.mean(NDVI)
    mean_EVI = np.mean(EVI)
    mean_SAVI = np.mean(SAVI)
    mean_GLI = np.mean(GLI)
    mean_NDRE = np.mean(NDRE)
    mean_GNDVI = np.mean(GNDVI)
    mean_NGRDI = np.mean(NGRDI)

    return mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI

In [ ]:
# Load the trained model
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/Competition/Submitted_models/cnn_regressionwithmoreinfo_modelNDVIfea_v2_S22.h5')
encoder = joblib.load('/content/drive/MyDrive/Competition/Submitted_models/location_encoder.pkl')

results = []
for i in range(len(Test_All_Path)):
    image_path = Test_All_Path[i]   # Assuming Val_All_Path contains paths to the image files
    new_image = average_images(image_path)
    mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI = compute_vegetation_indices(new_image);
    # new_image = np.stack([R, G, B, NIR, RE, DB, NDVI, EVI, SAVI, GNDVI, GLI, NGRDI, NDRE], axis=-1)
    new_images = np.expand_dims(new_image, axis=0)  # Add batch dimension

    location = Test_All_loc[i]      # Location data
    loc_encoded = encoder.transform(np.array(location).reshape(-1, 1))

    nitrogen = Test_All_Ninfo[i]    # Nitrogen data
    nitrogen = np.array(nitrogen).reshape(1, 1)  # Ensure nitrogen has shape (1, 1)

    mean_NDVI = np.array(mean_NDVI).reshape(1, 1)
    mean_EVI = np.array(mean_EVI).reshape(1, 1)
    mean_SAVI = np.array(mean_SAVI).reshape(1, 1)
    mean_GLI = np.array(mean_GLI).reshape(1, 1)
    mean_NDRE = np.array(mean_NDRE).reshape(1, 1)
    mean_GNDVI = np.array(mean_GNDVI).reshape(1, 1)
    mean_NGRDI = np.array(mean_NGRDI).reshape(1, 1)


    new_data = (new_images, loc_encoded, nitrogen,  mean_NDVI, mean_EVI, mean_SAVI, mean_GLI, mean_NDRE, mean_GNDVI, mean_NGRDI)

    # Predict using the model
    predictions = model.predict(new_data)

    results.append({
        'File Path': image_path[0],
        'Predicted Yield': predictions[0][0]
    })

    # Print or process predictions
    print(f'Predicted yield: {predictions[0][0]}')

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
excel_file = 'predictions.xlsx'
df.to_excel(excel_file, index=False)


# Download the file
files.download(excel_file)

print(f'Predictions saved to {excel_file}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted yield: 145.54908752441406
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted yield: 168.83538818359375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted yield: 163.3954620361328
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 157.00303649902344
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 162.02182006835938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted yield: 166.75228881835938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 137.23617553710938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 185.60952758789062
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 165.8933868408203
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted yield: 147.8783721923828
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted yield: 134.43313598632812
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted yield: 164.07972717285156
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted yield: 163.66212463378906
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predictions saved to predictions.xlsx
